# Bucketize

In [1]:
import cudf
import nvtabular as nvt
import numpy as np

In [2]:
students = cudf.DataFrame(
    data={'student_id': np.arange(1_000),
          'age': np.random.randint(1, 19, 1_000)
})
students.head()

,student_id,age
0,0,17
1,1,5
2,2,10
3,3,13
4,4,9


Let us bucketize the `age` column into :

* toddler: 1-3
* preschooler: 4-5
* school-aged: 6-12
* teenager: 13-18

In [3]:
out = ['age'] >> nvt.ops.Bucketize(boundaries={'age': [4, 6, 13]}) >> nvt.ops.Rename(name='age_bucketized')

wf = nvt.Workflow(['age'] + out)
ds = nvt.Dataset(students)

ds_out = wf.fit_transform(ds)
ds_out.compute().head()

,age_bucketized,age
0,3,17
1,1,5
2,2,10
3,3,13
4,2,9


In [4]:
ds_out.compute().groupby('age_bucketized')['age'].unique()

age_bucketized
0                   [1, 2, 3]
1                      [4, 5]
2    [6, 7, 8, 9, 10, 11, 12]
3    [13, 14, 15, 16, 17, 18]
Name: age, dtype: list